## Requirements
- `astropy.io.fits` to access the fits files
- `matplotlib.pyplot` for plotting
- `numpy` for array handling
- `os` for file directory finding

In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import os
%matplotlib qt

# ONLY WORKS ON MY LAPTOP, COULDN'T GET THE PATH TO WORK FOR THE REMOTE REPO
direc = r"C:\Users\rober\OneDrive\Documents\_Docs\Year 4\TGP\TGP_Asteroids2_2025"

 Useful `.header` keys:
- RA
- DEC 
- AZIMUTH
- ALTITUDE
- FILTER 
- EXPOSURE

## Cropping
- open and display first image from night 1 to see the border, and crop until the border disappears

In [67]:
# READ FIRST IMAGE
image = fits.open(direc + r'\2025-10-09\PIRATE_165659_OSL_ROE_Asteroids_2_00_Standard_Star_1_00_Filter_R_00_2025_10_09_19_46_12.fits')[0].data

# REMOVE BORDER
image = image[21:2027,21:2027]

# PLOT
fig,ax = plt.subplots(figsize=(8,8))
ax.imshow(image, cmap='gray', vmax=1500)
plt.show()

- Crop all files from all nights, and all calibration frames

In [ ]:
# EXTRACT ALL SCIENCE FRAMES
fnames_day1 = os.listdir(direc + r'\2025-10-09')
files_day1 = [direc + '\\2025-10-09\\' + file for file in fnames_day1]
fnames_day2 = os.listdir(direc + r'\2025-10-19')
files_day2 = [direc + '\\2025-10-19\\' + file for file in fnames_day2]
fnames_day3 = os.listdir(direc + r'\2025-10-22')
files_day3 = [direc + '\\2025-10-22\\' + file for file in fnames_day3]

# EXTRACT ALL CALIBRATION FRAMES
cnames_day1 = os.listdir(direc + r'\Calibrations_1')
calibs_day1 = [direc + '\\Calibrations_1\\' + file for file in cnames_day1]
cnames_day2 = os.listdir(direc + r'\Calibrations_2')
calibs_day2 = [direc + '\\Calibrations_2\\' + file for file in cnames_day2]
cnames_day3 = os.listdir(direc + r'\Calibrations_3')
calibs_day3 = [direc + '\\Calibrations_3\\' + file for file in cnames_day3]

# COMBINE ALL FRAMES
all_names = fnames_day1 + fnames_day2 + fnames_day3 + cnames_day1 + cnames_day2 + cnames_day3
all_files = files_day1 + files_day2 + files_day3 + calibs_day1 + calibs_day2 + calibs_day3

# CROP ALL AND GIVE KEY AS EXPOSURE CODE (E.G. 162431), SHAPES WILL BE (2006,2006)
cropped = {}
headers = {}
for (name, file) in zip(all_names, all_files):
    cropped[name[7:13]] = fits.open(file)[0].data[21:2027,21:2027]
    headers[name[7:13]] = fits.open(file)[0].header

# CHECK ALL CROPPED IMAGES HAVE THE CORRECT SHAPE
for key in cropped:
    if cropped[key].shape != (2006,2006):
        print(key)

### Bias

In [ ]:
# GET MEAN BIAS FRAME FOR EACH NIGHT
bias_day1 = (cropped["164291"] +
             cropped["164292"] +
             cropped["164293"] +
             cropped["164294"])/4
bias_day2 = (cropped["165895"] +
             cropped["165896"] +
             cropped["165897"] +
             cropped["165898"])/4
bias_day3 = (cropped["166960"] +
             cropped["166961"] +
             cropped["166962"] +
             cropped["166963"])/4

# BIAS-SUBTRACT THE DARK FRAMES
dark1_bs_60s = (cropped["164277"] +
                cropped["164278"] +
                cropped["164279"] +
                cropped["164280"])/4 - bias_day1
dark2_bs_60s = (cropped["165881"] +
                cropped["165882"] +
                cropped["165883"] +
                cropped["165884"])/4 - bias_day2
dark3_bs_60s = (cropped["166946"] +
                cropped["166947"] +
                cropped["166948"] +
                cropped["166949"])/4 - bias_day2

2.572754132915312


In [ ]:
fig,axs = plt.subplots(2,2,figsize=(12,12))
axs[0,0].imshow(cropped["166095"], cmap='plasma', vmin=27000, vmax=28000)
axs[0,1].imshow(cropped["166096"], cmap='plasma', vmin=33000, vmax=36000)
axs[1,0].imshow(cropped["166097"], cmap='plasma', vmin=34000, vmax=36000)
axs[1,1].imshow(cropped["166098"], cmap='plasma', vmin=33000, vmax=36000)
fig.tight_layout(); plt.show()

print(headers["166095"]["EXPOSURE"])
print(headers["166096"]["EXPOSURE"])
print(headers["166097"]["EXPOSURE"])
print(headers["166098"]["EXPOSURE"])

3.9300000667572
5.67000007629395
6.40999984741211
7.23000001907349


240.0
